In [1]:
from os.path import join, dirname, exists
from os import makedirs
from itertools import chain, islice
from thesis_v2 import dir_dict
from thesis_v2.configs.model.maskcnn_polished_with_rcnn_k_bl import multipath_models_8k_generator
from thesis_v2.analysis.io import collect_rcnn_k_bl_source_analysis
from thesis_v2.analysis.imagenet8k import load_ccmax

In [2]:
import pandas as pd

In [3]:
def load_new_result():
    f_main_result = join(dir_dict['analyses'], 'imagenet_8k', 'multipath', 'source_analysis.pkl')
    if not exists(f_main_result):
        makedirs(dirname(f_main_result), exist_ok=True)
        df_main_result = collect_rcnn_k_bl_source_analysis(
            fixed_keys = {
            'kernel_size_l23': 3,
            'kernel_size_l1': 9,
            'ff_1st_block': True,
            'pooling_ksize': 3,
            'pooling_type': 'avg',
            'bn_after_fc': False,
            'rcnn_bl_psize': 1,
            'rcnn_bl_ptype': None,
            'input_size': 50,
            'split_seed': 'legacy',
            'dataset_prefix': 'yuanyuan_8k_a_3day',
            'model_prefix': 'maskcnn_polished_with_rcnn_k_bl',
            'scale_name': '0.01',
            'scale': '0.01',
            'smoothness_name': '0.000005',
            'smoothness': '0.000005',
            'multi_path': True,
            'multi_path_separate_bn': True,
        },
            generator=chain(
                multipath_models_8k_generator(with_source=True),
            ),
            total_num_param=28,
            train_size_mapping={
                None: 5120,
            },
            no_missing_data=False,
        )
        df_main_result.to_pickle(f_main_result)
        del df_main_result
    df_main_result = pd.read_pickle(f_main_result)
    return df_main_result

In [4]:
df_main_result_new = load_new_result()
df_main_result_new

num_param  \
act_fn   ff_1st_bn_before_act loss_type model_seed num_layer out_channel rcnn_bl_cls readout_type train_keep              
relu     False                mse       0          2         8           2           cm-avg       1280            18085   
                                                                                                  2560            18085   
                                                                                                  5120            18085   
                                                                                     cm-last      1280            18085   
                                                                                                  2560            18085   
...                                                                                                                 ...   
softplus True                 poisson   1          3         32          7           inst-avg     2560               -1   
                                                                                                  5120               -1   
                                                                                     inst-last    1280               -1   
                                                                                                  2560               -1   
                                                                                                  5120               -1   

                                                                                                                                                source_analysis  
act_fn   ff_1st_bn_before_act loss_type model_seed num_layer out_channel rcnn_bl_cls readout_type train_keep                                                     
relu     False                mse       0          2         8           2           cm-avg       1280        {('I', 'B1'): 2.4155292165636055, ('I', 'B1', ...  
                                                                                                  2560        {('I', 'B1'): 3.034409683792191, ('I', 'B1', '...  
                                                                                                  5120        {('I', 'B1'): 3.214955942617024, ('I', 'B1', '...  
                                                                                     cm-last      1280        {('I', 'B1'): 2.4735672743749433, ('I', 'B1', ...  
                                                                                                  2560        {('I', 'B1'): 3.1474041177030188, ('I', 'B1', ...  
...                                                                                                                                                         ...  
softplus True                 poisson   1          3         32          7           inst-avg     2560                                                     None  
                                                                                                  5120                                                     None  
                                                                                     inst-last    1280                                                     None  
                                                                                                  2560                                                     None  
                                                                                                  5120                                                     None  

[6912 rows x 2 columns]

In [5]:
# disable what's below. it's there just for checking against original results generated piece by piece.

In [6]:
# validate against old result
def load_old_result():
    ret = pd.read_pickle(join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201114+20201118_separatebn', 'source_analysis.pkl')).sort_index()
    assert ret.index.is_unique
    return ret

In [7]:
df_main_result_old = load_old_result()

In [8]:
df_main_result_old

num_param  \
act_fn   ff_1st_bn_before_act loss_type model_seed num_layer out_channel rcnn_bl_cls readout_type train_keep              
relu     False                mse       0          2         8           2           cm-avg       1280            18085   
                                                                                                  2560            18085   
                                                                                                  5120            18085   
                                                                                     cm-last      1280            18085   
                                                                                                  2560            18085   
...                                                                                                                 ...   
softplus True                 poisson   1          3         32          9           cm-avg       2560               -1   
                                                                                                  5120               -1   
                                                                         10          cm-avg       1280               -1   
                                                                                                  2560               -1   
                                                                                                  5120               -1   

                                                                                                                                                source_analysis  
act_fn   ff_1st_bn_before_act loss_type model_seed num_layer out_channel rcnn_bl_cls readout_type train_keep                                                     
relu     False                mse       0          2         8           2           cm-avg       1280        {('I', 'B1'): 2.4155292165636055, ('I', 'B1', ...  
                                                                                                  2560        {('I', 'B1'): 3.034409683792191, ('I', 'B1', '...  
                                                                                                  5120        {('I', 'B1'): 3.214955942617024, ('I', 'B1', '...  
                                                                                     cm-last      1280        {('I', 'B1'): 2.4735672743749433, ('I', 'B1', ...  
                                                                                                  2560        {('I', 'B1'): 3.1474041177030188, ('I', 'B1', ...  
...                                                                                                                                                         ...  
softplus True                 poisson   1          3         32          9           cm-avg       2560                                                     None  
                                                                                                  5120                                                     None  
                                                                         10          cm-avg       1280                                                     None  
                                                                                                  2560                                                     None  
                                                                                                  5120                                                     None  

[7776 rows x 2 columns]

In [9]:
def check_old_vs_new():
    result = df_main_result_new.index.isin(df_main_result_old.index)
    assert result.all()
    df_main_result_old_sliced = df_main_result_old[df_main_result_old.index.isin(df_main_result_new.index)]
    assert df_main_result_old_sliced.equals(df_main_result_new)

In [10]:
check_old_vs_new()